In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [7]:
sfrData_withUFLS = np.genfromtxt(r"insert directory of the text file of dataset",skip_header=1)[:,1:]; #data

In [8]:
x=np.unique(sfrData_withUFLS[:,[0,2,3,9,13]],axis=0)[:,0:4];
y=np.unique(sfrData_withUFLS[:,[0,2,3,9,13]],axis=0)[:,4]
y_bi=y.copy();
y_bi[y > 0] = 1;
y_bi[y == 0] = 0;

train_ratio = 0.80
test_ratio = 0.2
trainX, testX, trainY_bi, testY_bi, trainY, testY = train_test_split(x, y_bi, y, test_size= 1 - train_ratio, random_state=42)

In [9]:
#Logistic regression splitting zero from nonzero UFLS

lr_model=LogisticRegression().fit(trainX,trainY_bi)
print("w=",lr_model.coef_,"\nb=",lr_model.intercept_,"\ntraining error=",lr_model.score(trainX,trainY_bi))
print("Cross validation=",lr_model.score(testX,testY_bi))

w= [[ 0.22090454 -0.04888795  7.93802714 -0.51636844]] 
b= [0.34501868] 
training error= 0.9754261678104937
Cross validation= 0.9761061946902655


In [10]:
#Logistic regression splitting nonzero UFLS into lower and higher than threshold

x_reg = trainX[~np.all(trainY.reshape(-1,1) == 0 ,axis=1)]
y_reg = trainY[~np.all(trainY.reshape(-1,1) == 0 ,axis=1)]

split_th=2.8

y_bi_l2=y_reg.copy()
y_bi_l2[y_reg<split_th]=0
y_bi_l2[y_reg>split_th]=1

lr_model_l2=LogisticRegression().fit(x_reg,y_bi_l2)
y_bi_l2_ALR=lr_model_l2.predict(x_reg)
print(lr_model_l2.score(x_reg,y_bi_l2))
print("w=",lr_model_l2.coef_,"\nb=",lr_model_l2.intercept_,"\ntraining error=",lr_model_l2.score(x_reg,y_bi_l2))
   
x_under3=x_reg[np.all(y_reg.reshape(-1,1) < split_th ,axis=1)]
y_under3=y_reg[np.all(y_reg.reshape(-1,1) < split_th ,axis=1)]

x_up3=x_reg[np.all(y_reg.reshape(-1,1) > split_th ,axis=1)]
y_up3=y_reg[np.all(y_reg.reshape(-1,1) > split_th ,axis=1)]

0.9203832177073009
w= [[ 0.00756986 -0.02628975  3.9091718  -0.38944138]] 
b= [0.07934798] 
training error= 0.9203832177073009


In [13]:
#Linear regression to predict values in each leaf

reg_model_l2_1=LinearRegression().fit(x_under3,y_under3)
pred_l2_under3=reg_model_l2_1.predict(x_under3)
reg_model_l2_1.score(x_under3,y_under3)


reg_model_l2_2=LinearRegression().fit(x_up3,y_up3)
pred_l2_up3=reg_model_l2_2.predict(x_up3)
reg_model_l2_2.score(x_up3,y_up3)


print(mean_absolute_error(y_under3, reg_model_l2_1.predict(x_under3)),mean_absolute_error(y_up3, reg_model_l2_2.predict(x_up3)))
print("l1=",reg_model_l2_1.coef_)
print("l2=",reg_model_l2_2.coef_)
print("c1=",reg_model_l2_1.intercept_)
print("c2=",reg_model_l2_2.intercept_)

0.04966747134082599 0.26343385894657223
l1= [ 1.83061649e-02 -1.40835874e-05  1.04587013e-01 -5.32954191e-02]
l2= [ 0.03373105 -0.00222222  0.83007189 -0.18424064]
c1= 0.0867690341003855
c2= 0.15574613422977546
